In [407]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd

fl = list(glob.glob('../features/*.pkl'))
for l in sorted(fl):
    print("'{}'".format(l))

'../features/cnn_1_aug1_feat.pkl'
'../features/cnn_1_aug_1_fold8_feat.pkl'
'../features/cnn_1_aug_rescale_preprocess_feat.pkl'
'../features/cnn_1_aug_skimage_denoise_feat.pkl'
'../features/cnn_1_aug_skimage_preprocess_feat.pkl'
'../features/cnn_1_feat.pkl'
'../features/cnn_2_aug1_feat.pkl'
'../features/cnn_2_aug_1_fold8_feat.pkl'
'../features/cnn_2_aug_denoise_preprocess_feat.pkl'
'../features/cnn_2_aug_rescale_preprocess_feat.pkl'
'../features/cnn_2_aug_skimage_preprocess_feat.pkl'
'../features/cnn_2_feat.pkl'
'../features/cnn_3_aug1_feat.pkl'
'../features/cnn_3_aug_1_fold8_feat.pkl'
'../features/cnn_3_aug_denoise_preprocess_feat.pkl'
'../features/cnn_3_aug_rescale_preprocess_feat.pkl'
'../features/cnn_3_aug_skimage_preprocess_feat.pkl'
'../features/cnn_4_aug1_feat.pkl'
'../features/cnn_4_aug1_feat_add_early.pkl'
'../features/cnn_4_aug_1_fold8_feat.pkl'
'../features/cnn_4_aug_denoise_preprocess_feat.pkl'
'../features/cnn_4_aug_rescale_preprocess_feat.pkl'
'../features/cnn_4_aug_skimag

In [408]:
def load_x_y():
    train_x,test_x=[],[]
    for f in fl:
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

(1604, 76) (8424, 76) (1604,)


In [409]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        params = {
                'colsample_bytree': 0.75,
                'colsample_bylevel':0.95,
                'gamma':0.2,
                'subsample': 0.9,
                'eta': 0.07,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight': 0.85
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(test_x)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 500, watchlist, 
                        early_stopping_rounds=20,
                        verbose_eval=1000)
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict(d_valid))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            w_test_pred = m.predict(d_test)*curr_r_val_loss
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred
        print(submission.head())
        submission.to_csv('../results/all_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=w_test_pred
        print(submission.head())
        submission.to_csv('../results/weighted_all_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)

    
print('def done')

def done


In [410]:
cv_test(3,42,True)

[0]	train-logloss:0.637142	valid-logloss:0.640334
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[145]	train-logloss:0.031117	valid-logloss:0.132949

[0]	train-logloss:0.635534	valid-logloss:0.639735
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[67]	train-logloss:0.069549	valid-logloss:0.151995

[0]	train-logloss:0.636248	valid-logloss:0.637373
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[73]	train-logloss:0.066246	valid-logloss:0.145008

local average valid loss 0.144055697105 val loss std 0.00799505359235
         id  is_iceberg
0  5941774d    0.026785
1  4023181e    0.751387
2  b20200e4    0.034

In [411]:
cv_test(5,42,True)

[0]	train-logloss:0.636025	valid-logloss:0.639621
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[97]	train-logloss:0.054248	valid-logloss:0.144529

[0]	train-logloss:0.63698	valid-logloss:0.63786
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[147]	train-logloss:0.03949	valid-logloss:0.103285

[0]	train-logloss:0.635998	valid-logloss:0.639716
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[72]	train-logloss:0.068445	valid-logloss:0.161313

[0]	train-logloss:0.637264	valid-logloss:0.639129
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-lo

In [412]:
cv_test(10,42,True)

[0]	train-logloss:0.636301	valid-logloss:0.639394
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[93]	train-logloss:0.059343	valid-logloss:0.15878

[0]	train-logloss:0.636017	valid-logloss:0.64001
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[128]	train-logloss:0.044419	valid-logloss:0.110522

[0]	train-logloss:0.636868	valid-logloss:0.63784
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[122]	train-logloss:0.047572	valid-logloss:0.107897

[0]	train-logloss:0.637071	valid-logloss:0.639347
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-l

In [413]:
cv_test(3,233,True)

[0]	train-logloss:0.637752	valid-logloss:0.637474
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[128]	train-logloss:0.041202	valid-logloss:0.12349

[0]	train-logloss:0.636489	valid-logloss:0.640258
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[127]	train-logloss:0.040492	valid-logloss:0.129247

[0]	train-logloss:0.635214	valid-logloss:0.639892
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[66]	train-logloss:0.058323	valid-logloss:0.175859

local average valid loss 0.146088296023 val loss std 0.0246208564683
         id  is_iceberg
0  5941774d    0.025587
1  4023181e    0.768483
2  b20200e4    0.0413

In [414]:
cv_test(5,233,True)

[0]	train-logloss:0.635663	valid-logloss:0.637974
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[96]	train-logloss:0.055337	valid-logloss:0.142495

[0]	train-logloss:0.637081	valid-logloss:0.635438
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[168]	train-logloss:0.032298	valid-logloss:0.07434

[0]	train-logloss:0.636114	valid-logloss:0.63931
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[93]	train-logloss:0.058749	valid-logloss:0.135662

[0]	train-logloss:0.636061	valid-logloss:0.642584
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-l

In [415]:
cv_test(10,233,True)

[0]	train-logloss:0.636652	valid-logloss:0.640064
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[144]	train-logloss:0.039425	valid-logloss:0.127594

[0]	train-logloss:0.636097	valid-logloss:0.638326
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[160]	train-logloss:0.032789	valid-logloss:0.148769

[0]	train-logloss:0.637083	valid-logloss:0.636985
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[107]	train-logloss:0.055064	valid-logloss:0.088375

[0]	train-logloss:0.637238	valid-logloss:0.638252
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until val